In [54]:
import pyomo as pyo
from pyomo.environ import *
import numpy as np
from datetime import datetime, timedelta
import pandas as pd

In [55]:
shift_output = pd.read_csv("Shift_Scheduling_output.csv")
start_time =[]
for i in range(len(shift_output)):
    start_time.append(datetime.strptime(shift_output['start'].iloc[i],'%Y-%m-%dT%H:%M:%S.%f'))
    print(start_time[i].time()) 
    


05:45:00
13:15:00
20:30:00
05:45:00
14:45:00
21:30:00
06:15:00
14:45:00
21:30:00
06:15:00
14:45:00
21:30:00
06:15:00
14:45:00
21:30:00
06:45:00
14:45:00
21:30:00
06:45:00
14:45:00
21:30:00


In [65]:
shift_output

,Unnamed: 0,Shift_name,staff,start_num,end_num,len,start,end
0,0,Shift-156,4,13,27,14,2023-01-01T05:45:00.000,2023-01-01T14:00:00.000
1,1,Shift-170,4,27,41,14,2023-01-01T13:15:00.000,2023-01-01T21:15:00.000
2,2,Shift-561,4,41,61,20,2023-01-01T20:30:00.000,2023-01-02T06:30:00.000
3,3,Shift-440,4,61,79,18,2023-01-02T05:45:00.000,2023-01-02T15:30:00.000
4,4,Shift-42,4,79,91,12,2023-01-02T14:45:00.000,2023-01-02T22:15:00.000
5,5,Shift-455,4,91,109,18,2023-01-02T21:30:00.000,2023-01-03T07:00:00.000
6,6,Shift-458,4,109,127,18,2023-01-03T06:15:00.000,2023-01-03T15:30:00.000
7,7,Shift-64,4,127,139,12,2023-01-03T14:45:00.000,2023-01-03T22:15:00.000
8,8,Shift-473,4,139,157,18,2023-01-03T21:30:00.000,2023-01-04T07:00:00.000
9,9,Shift-476,4,157,175,18,2023-01-04T06:15:00.000,2023-01-04T15:30:00.000


In [56]:
# Initialize the model
model = ConcreteModel()

## Define the Sets

### Employees

In [57]:
np.random.seed(0)

# Generate unique numbers to be used for the all_employees list
unique_numbers = np.random.choice(range(10000, 99999), size=84, replace=False)
# Generates a list of employees with randomly generated empIDs 
all_employees = ['emp{}'.format(number) for number in unique_numbers]

# And a list of employee IDs who desire certain shift patterns
employees_with_desirable_shifts = np.random.choice(all_employees, size=30, replace=False)

# Shift types
ShiftTypes = ['D', 'E', 'N', 'O']

#NOTE: I guess here we have to input the values from the model of Lorenzo and Joachim
#Define a dictionary with the shifts for each day of the week
shifts_by_day = {
    'Monday': ['D', 'E', 'N', 'O'],
    'Tuesday': ['D', 'E', 'N', 'O'],
    'Wednesday': ['D', 'E', 'N', 'O'],
    'Thursday': ['D', 'E', 'N', 'O'],
    'Friday': ['D', 'E', 'N', 'O'],
    'Saturday': ['D', 'E', 'N', 'O'],
    'Sunday': ['D', 'E', 'N', 'O']  
} # Need input from Lorenzo and Joachim here

Weeks = range(12)
# Half-hour periods (24 hours * 2)
time_periods = list(range(48))

# minimum and ideal staffing levels


In [58]:
# Define the set of all employees
model.Employees = Set(initialize=all_employees)

# Define the subset of employees with desirable shift patterns
model.Employees_DES = Set(within=model.Employees, initialize=employees_with_desirable_shifts)
#Note: Need to connect the employees who has desirable shift patterns to the shift patterns that they desire

# Define the complete set of shifts
model.ShiftTypes = Set(initialize=ShiftTypes)

# Define subsets for the shift types
model.Shifts_W = Set(within=model.Shifts, initialize=['D', 'E', 'N'])  # Work shifts (Day, Evening, Night)
model.Shifts_O = Set(within=model.Shifts, initialize=['O'])  # Off shifts 



In [76]:
e = []
for i in range(12,48):
    e.append(i)
print(len(e), 48-12)

36 36


In [69]:
shifts = []
for t in time_periods:
    for duration in range(12,48):
        for duration in range(12,20):
            shifts.append((t, duration))
            
shifts

[(0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),


In [68]:
def is_shift_cov(shift,period):
    interval = [(shift[0],  min(shift[0] + shift[1], 48))]
    if shift [0] + shift  >= 48:
        intervals.append
    
    start = shift[0]
    end = shift[0] + shift[1]
    if end < 48: 
        return period >= start and period < end
    elif end >= 48:
        return  period <= start and period > end
    


In [59]:
##all_possible_shifts = #input from Joachim and Lorenzo

### Shifts

In [60]:
model.Shifts_W.display()

Shifts_W : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 : Shifts :    3 : {'D', 'E', 'N'}


### Shift Days

In [61]:
# Define the days of the week set
model.Days = Set(initialize=shifts_by_day.keys())

# Define the set of shifts for each day of the week, which must be contained in the set S of the model
model.Shift_days = Set(model.Days, within=model.Shifts, initialize=lambda model, day: shifts_by_day[day])

# Loop through
for day in model.Days:
    print(f'Shifts on {day}:', list(model.Shift_days[day].data()))

Shifts on Monday: ['D', 'E', 'N', 'O']
Shifts on Tuesday: ['D', 'E', 'N', 'O']
Shifts on Wednesday: ['D', 'E', 'N', 'O']
Shifts on Thursday: ['D', 'E', 'N', 'O']
Shifts on Friday: ['D', 'E', 'N', 'O']
Shifts on Saturday: ['D', 'E', 'N', 'O']
Shifts on Sunday: ['D', 'E', 'N', 'O']


### Weeks

In [62]:
# Initialize the weeks in the planning horizon
model.Weeks = Set(initialize=Weeks)

# Initialize the days in each week w
model.Week_days = Set(model.Weeks, initialize=lambda model, w: shifts_by_day.keys())


In [63]:
model.Week_days.display()

Week_days : Size=12, Index=Weeks, Ordered=Insertion
    Key : Dimen : Domain : Size : Members
      1 :     1 :    Any :    7 : {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'}
      2 :     1 :    Any :    7 : {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'}
      3 :     1 :    Any :    7 : {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'}
      4 :     1 :    Any :    7 : {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'}
      5 :     1 :    Any :    7 : {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'}
      6 :     1 :    Any :    7 : {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'}
      7 :     1 :    Any :    7 : {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'}
      8 :     1 :    Any :    7 : {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'}
  

### Days

In [64]:
# Define your planning horizon
start_date = datetime(2023, 1, 1)  # Example start date (year, month, day)
num_weeks = 12  # Number of weeks in your planning horizon

# Generate a list of Sundays within the planning horizon
sundays = [(start_date + timedelta(days=7 * week)).isoformat() for week in range(num_weeks)]

# Define the set of Sundays in the planning horizon
model.Days_SUN = Set(initialize=sundays)

# Access the set elements to verify
for sunday in model.Days_SUN:
    print(f'Sunday date: {sunday}')

Sunday date: 2023-01-01T00:00:00
Sunday date: 2023-01-08T00:00:00
Sunday date: 2023-01-15T00:00:00
Sunday date: 2023-01-22T00:00:00
Sunday date: 2023-01-29T00:00:00
Sunday date: 2023-02-05T00:00:00
Sunday date: 2023-02-12T00:00:00
Sunday date: 2023-02-19T00:00:00
Sunday date: 2023-02-26T00:00:00
Sunday date: 2023-03-05T00:00:00
Sunday date: 2023-03-12T00:00:00
Sunday date: 2023-03-19T00:00:00


### Time periods

In [12]:

# Define the set of half-hour periods within each day
model.Time = Set(initialize=time_periods)

In [13]:
# Assuming 'model' is your Pyomo model
model.Time.display()


Time : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   48 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47}
